In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'g:\\Projects\\Skin-Disease-Classification\\Skin-Disease-Classification'

In [88]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingModelConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float




In [89]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [90]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_model_config(self) -> TrainingModelConfig:
        training = self.config.training_model
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"sort-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_model_config = TrainingModelConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
        )
        
        return training_model_config


    
    

In [91]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

tf.config.run_functions_eagerly(True)


In [92]:
class TrainingModel:
    def __init__(self, config: TrainingModelConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=[
                tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall'),
                tf.keras.metrics.AUC(name='auc'),
                tf.keras.metrics.TopKCategoricalAccuracy(k=5, name='top_k_categorical_accuracy')
            ]
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            class_mode="categorical", 
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

    

In [ ]:

try:
    config = ConfigurationManager()
    training_config = config.get_training_model_config()
    training = TrainingModel(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e


[2025-03-16 22:33:06,295: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-16 22:33:06,300: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-16 22:33:06,301: INFO: common: created directory at: artifacts]
[2025-03-16 22:33:06,405: INFO: common: created directory at: artifacts\training_model]
[2025-03-16 22:33:06,763: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 2000 images belonging to 7 classes.
Found 8015 images belonging to 7 classes.


c:\Users\Asus\anaconda3\envs\skin-disease-env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25


c:\Users\Asus\anaconda3\envs\skin-disease-env\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


250/250 ━━━━━━━━━━━━━━━━━━━━ 4257s 17s/step - accuracy: 0.6452 - auc: 0.8789 - loss: 1.1725 - precision: 0.7458 - recall: 0.5503 - top_k_categorical_accuracy: 0.9583 - val_accuracy: 0.7092 - val_auc: 0.9461 - val_loss: 0.8059 - val_precision: 0.8814 - val_recall: 0.5766 - val_top_k_categorical_accuracy: 0.9869
Epoch 2/25
  1/250 ━━━━━━━━━━━━━━━━━━━━ 1:08:08 16s/step - accuracy: 0.6250 - auc: 0.9138 - loss: 1.0052 - precision: 0.9412 - recall: 0.5000 - top_k_categorical_accuracy: 0.9688

c:\Users\Asus\anaconda3\envs\skin-disease-env\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


250/250 ━━━━━━━━━━━━━━━━━━━━ 259s 973ms/step - accuracy: 0.6250 - auc: 0.9138 - loss: 1.0052 - precision: 0.9412 - recall: 0.5000 - top_k_categorical_accuracy: 0.9688 - val_accuracy: 0.7122 - val_auc: 0.9467 - val_loss: 0.8084 - val_precision: 0.8848 - val_recall: 0.5766 - val_top_k_categorical_accuracy: 0.9874
Epoch 3/25
106/250 ━━━━━━━━━━━━━━━━━━━━ 36:20 15s/step - accuracy: 0.7014 - auc: 0.9358 - loss: 0.8668 - precision: 0.8241 - recall: 0.6027 - top_k_categorical_accuracy: 0.9845